In [16]:
import iris
import time
import os

## IRIS database operations

In [17]:
username = 'demo'
password = 'demo'
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"{hostname}:{port}/{namespace}"

In [18]:
print(CONNECTION_STRING)

localhost:1972/USER


In [19]:
conn = iris.connect(CONNECTION_STRING, username, password)
cursor = conn.cursor()

## Creating a new table

In [20]:
table_name = "Papers.General_Data"
table_definition = """(
    title VARCHAR(255),
    doi VARCHAR(255) UNIQUE,
    pub_date TEXT,
    authorships TEXT,
    topics TEXT,
    cited_by INT,
    keywords TEXT,
    abstract TEXT,
    titleVector VECTOR(DOUBLE, 384),
    abstractVector VECTOR(DOUBLE, 384),
    topicsVector VECTOR(DOUBLE, 384),
    keywordsVector VECTOR(DOUBLE, 384)
)"""

In [79]:
interest_name = "Papers.User_Interests"
interest_definition = """(
    user_id VARCHAR(255),
    interests VARCHAR(255)
)"""

In [80]:
try:
    cursor.execute(f"DROP TABLE {interest_name}")  
except:
    pass
cursor.execute(f"CREATE TABLE {interest_name} {interest_definition}")

0

In [69]:
try:
    cursor.execute(f"DROP TABLE {table_name}")  
except:
    pass
cursor.execute(f"CREATE TABLE {table_name} {table_definition}")

0

In [21]:
def clean_title(title):
    max_length = 255

    if not title:
        return None
    
    cleaned_title = title[:max_length]

    try:
        return cleaned_title.encode("utf-8").decode("utf-8") 
    except UnicodeDecodeError:
        print(f"Invalid characters in title: {title}")
        return None  


In [22]:
def is_doi_unique(cursor, doi):
    """Check if DOI already exists in the database"""
    cursor.execute("SELECT COUNT(*) FROM Papers.GeneralData WHERE doi = ?", (doi,))
    count = cursor.fetchone()[0]
    return count == 0  


In [23]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import json

/Users/reaganrazon/Documents/Treehacks25/hackathon-2024/iris-env/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


## PANDAS IMPLEMENTATION

In [24]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [25]:
def upload_and_embed(filename, table_name, conn):
    """
    Reads JSON, processes embeddings, and uploads data into a SQL table.
    
    Parameters:
    - filename: str (path to JSON file)
    - table_name: str (SQL table name)
    - conn: Database connection object
    """

    with open(filename, "r", encoding="utf-8") as file:
        papers = json.load(file)

    df = pd.DataFrame(papers)

    # Clean and process columns
    df["Title"] = df["Title"].fillna("").apply(str)
    df["DOI"] = df["DOI"].fillna("").apply(str)
    df["Date"] = df["Date"]
    df["Authorships"] = df["Authorships"].apply(lambda x: "; ".join(x) if isinstance(x, list) else "")
    df["Topics"] = df["Topics"].apply(lambda x: "; ".join(x) if isinstance(x, list) else "")
    df["Cited by"] = df["Cited by"].fillna(0).astype(int)
    df["Keywords"] = df["Keywords"].apply(lambda x: "; ".join(kw["display_name"] for kw in x) if isinstance(x, list) else "")
    df["Abstract"] = df["Abstract"].fillna("").apply(str)

    # Generate embeddings for each text field
    print("Generating embeddings... This may take some time.")
    # df["TitleVector"] = df["Title"].apply(generate_embedding)
    # df["AbstractVector"] = df["Abstract"].apply(generate_embedding)
    # df["TopicsVector"] = df["Topics"].apply(generate_embedding)
    # df["KeywordsVector"] = df["Keywords"].apply(generate_embedding)

    df["TitleVector"] = model.encode(df["Title"].tolist(), batch_size=256, show_progress_bar=True, normalize_embeddings=True).tolist()
    df["AbstractVector"] = model.encode(df["Abstract"].tolist(), batch_size=256, show_progress_bar=True, normalize_embeddings=True).tolist()
    df["TopicsVector"] = model.encode(df["Topics"].tolist(), batch_size=256, show_progress_bar=True, normalize_embeddings=True).tolist()
    df["KeywordsVector"] = model.encode(df["Keywords"].tolist(), batch_size=256, show_progress_bar=True, normalize_embeddings=True).tolist()

    # Define SQL Insert Query (uses parameterized queries for security)
    sql = f"""INSERT INTO {table_name} (title, doi, pub_date, authorships, topics, cited_by, keywords, abstract,titleVector,abstractVector,topicsVector,keywordsVector) 
            VALUES (?, ?, ?, ?, ?, ?, ?, ?,TO_VECTOR(?),TO_VECTOR(?),TO_VECTOR(?),TO_VECTOR(?))"""

    start_time = time.time()

    # Convert DataFrame to List of Tuples for Bulk Insert
    data = df[["Title", "DOI", "Date", "Authorships", "Topics", "Cited by", "Keywords", "Abstract", 
               "TitleVector", "AbstractVector", "TopicsVector", "KeywordsVector"]].values.tolist()
    
    data = [
    (
        row['Title'], 
        row['DOI'], 
        row['Date'], 
        row['Authorships'], 
        row['Topics'], 
        row['Cited by'], 
        row['Keywords'], 
        row['Abstract'], 
        str(row['TitleVector']),
        str(row['AbstractVector']),
        str(row['TopicsVector']),
        str(row['KeywordsVector'])
    )
    for index, row in df.iterrows()
]
    

    # Insert into database using executemany (faster than looping)
    cursor.executemany(sql, data)
    end_time = time.time()

    print(f"Successfully added {len(df)} entries from {filename} in {end_time - start_time:.2f} seconds.")
    # return df  


In [44]:
datasets = ["thousand_papers4.json","thousand_papers5.json","thousand_papers6.json"]

datasets2 = ["filtered_paper_MAIN.json","filtered_paper_MAIN2.json","filtered_paper_MAIN3.json","filtered_paper_MAIN4.json","filtered_paper_MAIN5.json", "filtered_paper2.json", "filtered_papers_paginated_2.json",'filtered_papers_paginated_3.json',"filtered_papers_paginated_4.json","filtered_papers.json","filtered_papers_paginated.json","filtered_papers_paginated_7.json","filtered_papers_paginated_5.json", "filtered_papers_paginated_6.json"]

for d in datasets:
    upload_and_embed(d, table_name,conn)



Generating embeddings... This may take some time.


Batches: 100%|██████████| 40/40 [00:12<00:00,  3.30it/s]


Successfully added 10000 entries from thousand_papers4.json in 46.57 seconds.
Generating embeddings... This may take some time.


Batches: 100%|██████████| 40/40 [00:14<00:00,  2.80it/s]


Successfully added 10000 entries from thousand_papers5.json in 43.09 seconds.
Generating embeddings... This may take some time.


Batches: 100%|██████████| 40/40 [00:14<00:00,  2.68it/s]


Successfully added 10000 entries from thousand_papers6.json in 45.61 seconds.


# testing search

In [88]:
sql = f""" SELECT COUNT(*) FROM {table_name} """

cursor.execute(sql)
row_count = cursor.fetchone()[0]
print(f"Total rows in table: {row_count}")

Total rows in table: 32107


In [86]:
searchPhrase = "ballet"
searchVector = model.encode(searchPhrase, normalize_embeddings=True).tolist() 

In [87]:
sql = f"""
    SELECT TOP ? title, topics, abstract, 
        VECTOR_DOT_PRODUCT(abstractVector, TO_VECTOR(?)) * 0.5 +
        VECTOR_DOT_PRODUCT(titleVector, TO_VECTOR(?)) * 0.3 +
        VECTOR_DOT_PRODUCT(keywordsVector, TO_VECTOR(?)) * 0.2 AS relevance_score
    FROM {table_name}
    ORDER BY relevance_score DESC
"""


numberOfResults = 10

cursor.execute(sql, [numberOfResults, str(searchVector), str(searchVector), str(searchVector)])

results = cursor.fetchall()
for row in results:
    print(row)

("The Application of 'Improvisational Accompaniment' in Piano Textbooks for Music Majors in Chinese Higher Education", 'Educational Reforms and Innovations; Educational Technology and Pedagogy; Diverse Music Education Insights', 'This research evaluates the pedagogical application of "Improvisational Accompaniment" in piano textbooks within Chinese higher music education, highlighting its significance in enhancing students\' practical skills and professional prospects. The study examines the integration of foundational piano training with improvisation, the promotion of creative teaching methods, and the unique educational features of Chinese textbooks. Empirical analysis presents a multifaceted teaching approach, including collective instruction and targeted practice, to bolster students\' improvisational proficiency. The findings offer insights for global music educators on syllabus development and innovative teaching strategies.', '.12960514098709224883')
('Physical Activity and Pub

In [63]:
import iris
import ssl

In [71]:
def main():
    connection_string = "localhost:1972/USER"
    username = "demo"
    password = "demo"

    context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
    context.verify_mode=ssl.CERT_REQUIRED
    context.check_hostname = False
    context.load_verify_locations("c:/InterSystems/Certs/isc-cert.pem")

    connection = iris.connect(connection_string, username, password)

    # when finished, use the line below to close the connection
    # connection.close()

In [70]:
from fastapi import FastAPI

In [ ]:
from fastapi import FastAPI, Query
from sentence_transformers import SentenceTransformer

app = FastAPI()
model = SentenceTransformer('all-MiniLM-L6-v2') 

connection_string = "localhost:1972/USER"
username = "demo"
password = "demo"

conn = iris.connect(connection_string, username, password)
cursor = conn.cursor()

@app.get("/papers")
def get_papers(
    searchPhrase: str,
    page: int = Query(1, alias="page"), 
    per_page: int = Query(6, alias="per_page")
):
    """
    Fetch papers with semantic search and pagination.
    """
    searchVector = model.encode(searchPhrase, normalize_embeddings=True).tolist()

    offset = (page - 1) * per_page

    sql = f"""
        SELECT title, topics, abstract, 
            VECTOR_DOT_PRODUCT(abstractVector, TO_VECTOR(?)) * 0.5 +
            VECTOR_DOT_PRODUCT(titleVector, TO_VECTOR(?)) * 0.3 +
            VECTOR_DOT_PRODUCT(keywordsVector, TO_VECTOR(?)) * 0.2 AS relevance_score
        FROM {table_name}
        ORDER BY relevance_score DESC
        LIMIT {per_page} OFFSET {offset}
    """

    cursor.execute(sql, [str(searchVector), str(searchVector), str(searchVector)])

    results = cursor.fetchall()

    return {
        "papers": [
            {
                "title": row[0],
                "topics": row[1],
                "abstract": row[2],
            }
            for row in results
        ],
        "nextPage": page + 1 if len(results) == per_page else None,
        "hasMore": len(results) == per_page
    }